<a href="https://colab.research.google.com/github/UiinKim/NLP_First/blob/main/2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

통계기반기법(배치) <-> 추론기반기법(미니배치)
gpu를 사용해서 병렬로 들어감 -> 학습속도 향상



모델이 context 정보를 입력받아 각 단어의 출현 확률을 출력

In [1]:
#원핫벡터 사용
#각 단어 벡터들은 fully connected layer로 연결
import numpy as np

c=np.array([[1, 0, 0, 0, 0, 0, 0]]) #input
W=np.random.randn(7, 3) #weight
h=np.matmul(c, W) #hidden node
print(h)

[[-0.22436509  0.35787729 -0.87647891]]


In [ ]:
c=np.array([[1, 0, 0, 0, 0, 0, 0]])
W=np.random.randn(7, 3)
layer=MatMul(W)